In [ ]:
import ee
import geepy

In [ ]:
ee.Initialize()

In [ ]:
config = geepy.params.configParams('input.json')

ls = ee.FeatureCollection(config.params['lsTiles'])
tiles = [geepy.feature.getTiles(ls, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id'])
l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id'])
l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id'])

tileNames = [tiles[i][0] for i in range(len(tiles))]
tileCoords = [tiles[i][1].geometry().getInfo()['coordinates'] for i in range(len(tiles))]

In [ ]:
# Save to drive
def send2drive(img, coords, desc, driveFolder):
    task2Drive = ee.batch.Export.image.toDrive(
        image = img,
        folder = driveFolder,
        description = desc,
        maxPixels = 1e10,
        region = coords,
        scale = 30,
    )
    return task2Drive

In [ ]:
tasks = {}
taskStatus = {}

for year in range(1990, 2017):
    y = str(year)
    start_d = config.params['years'][y]+config.params['period']['dry']['start']
    end_d = config.params['years'][y]+config.params['period']['dry']['end']
    #print("Start-End dry season %s %s" %(start_d, end_d))
    
    if(year < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)
    elif(year in (2002,2011,2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)
    elif(year > 2002 and year < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)

    filt = filtered.median()
    
    lsClip = [geepy.image.clipImageCollection(filt, tiles[i][1]) for i in range(len(tiles))]
    
    #print('Cloud Cover and Shadow classification for year %s' %(y))
    cloudMask = [geepy.cloud.getFmask(lsClip[a]) for a in range(len(lsClip))]
    
    print('Config task for %s' %(y))
    tasks[str(y)] = [send2drive(cloudMask[i], tileCoords[i], 'cloudMask'+str(year)+'dry'+tileNames[i], 'cloudMask'+str(year)+'dry') for i in range(len(tiles))]
    
    taskStatus[str(y)] = [tasks[str(y)][i].status()['state'] for i in range(len(tiles))]

In [ ]:
import time

for i in tasks.keys():
    print('Exporting to drive: %s' %(i))
    [tasks[i][j].start() for j in range(len(tasks[i]))]

    taskPoolStatus = False
    
    while taskPoolStatus == False:
        taskStatus[i] = [tasks[i][k].status()['state'] for k in range(len(tasks[i]))]
        taskPoolStatus = all(status == 'COMPLETED' for status in taskStatus[i])
            
        if taskPoolStatus:
            print('Exporting data for year %s is completed.' %(i))
            print('Waiting 1 minute.')
            time.sleep(60)
            
            
#### implementar um timer para verificar quanto tempo demorou para exportar todas as imagens